In [1]:
import numpy as np

from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace, entanglement_of_formation
from qiskit.circuit import QuantumCircuit

import itertools


#@jit
def Bipartite_Negativity(rho ):
    
    matrix = rho.data
    #print(matrix)
    eigenvalues, eigenvectors = np.linalg.eig(matrix)

    return sum([ ( np.abs(eigenvalue) - eigenvalue )/2 for eigenvalue in eigenvalues ])
    
    
#@jit
def Multipartite_Negativity(circuit : QuantumCircuit, mode='max' ):

    #possible combinations: N choose 2
    #because choosing ij = ji ---> 
    #calculate [(N Choose 2) / 2] possible outcomes
    
    qubits_list = circuit.qubits
    rho = DensityMatrix(circuit)
    
    index_list = [qubit._index for qubit in qubits_list]

    num_q = len(qubits_list)

    negativity_dict = {}

    if num_q == 2:
        return Bipartite_Negativity(rho)

    else:
        qubits_list = circuit.qubits
    
        index_list = np.array([qubit._index for qubit in qubits_list])

        negativity_matrix = np.zeros(shape= (num_q, num_q))

        for qbit_indx1, ind_i in enumerate(index_list):
            
            for qbit_indx2, ind_j in enumerate(index_list[:ind_i]):
                
                qubit_trace_list =  [k for k in index_list if k not in (qbit_indx1, qbit_indx2)]
                rho_p = partial_trace(rho, qubit_trace_list)

                negativity_matrix[ind_i][ind_j] = Bipartite_Negativity(rho_p)
    
    if mode== 'max':
        return negativity_matrix.max()
    
    elif mode== 'total':
        return np.sum(negativity_matrix)


In [4]:

qc = QuantumCircuit(4)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(2, 3)

r = DensityMatrix(qc)

r.draw(output= 'latex')

print(Multipartite_Negativity(qc, mode= 'total'))

0.0


/tmp/ipykernel_7631/941773238.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
  negativity_matrix[ind_i][ind_j] = Bipartite_Negativity(rho_p)
